<a href="https://colab.research.google.com/github/choisangh/STUDY/blob/main/ML/%ED%8A%9C%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 튜닝(Tuinng)

## 개념


* 모델 학습 과정에서 최적의 하이퍼파라미터 집합을 찾는 것

## 종류
1. GridSearchCV
2. Optuna

### GridSearchCV<br>
ref: https://velog.io/@dlskawns/Machine-Learning-RandomizedSearchCV-GridSearchCV-%EC%A0%95%EB%A6%AC-%EC%8B%A4%EC%8A%B5<br>
ref: https://teddylee777.github.io/scikit-learn/grid-search-%EB%A1%9C-hyperparameter%EC%B5%9C%EC%A0%81%ED%99%94<br>
* 모델에서 사용할 hyperparmeter 값 리스트를 딕셔너리 형태로 각 변수에 입력한 후, 모든 hyperparameter 조합에 대한 학습을 진행한다.
* 사용자가 수동으로 하이퍼파라미터 조합을 찾는 시간을 줄여준다는 장점이 있다.
* 그러나 지정해준 hyperparameter 후보군 개수에 비례해 시간이 늘어나 조합을 찾을 때까지 시간이 매우 오래걸린다는 단점이 있다.

<font size=5> 사용 방법</font>
1. dictionary 변수에 사용할 hyperparmeter 값들을 저장한다.
2. GridSearchCV함수에 사용할 모델과, hyperparameter값들을 저장한 dictionary 변수를 전달한다.

<font size=5 >예시 데이터</font><br>
scikit-learn 패키지의 피마 당뇨병 데이터를 사용하였다.

<font size=5 ><b>Import Packages</b></font>

In [ ]:
from sklearn.model_selection import GridSearchCV

<font size=5 ><b> API: sklearn.model_selection.GridSearchCV</b></font><br>
* ref: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn-model-selection-gridsearchcv
<img src='https://drive.google.com/uc?export=download&id=11AyWLEi-ray3EYsL3u1QhwBXUe-kVtdr' width=800  height=100>

<font size=5 >parameter</font>
* <b>estimator</b>: 적용할 모델
* <b>param_grid</b>: gridsearch가 찾을 parameter 집합 정의
* <b>scoring</b>: 평가 기준
* <b>n_jobs</b>: 사용할 컴퓨터 코어의 개수. -1로 설정하면 모든 코어를 사용한다.
* <b>refit</b>: 최적의 파라미터로 재학습할지를 결정하는 것이다.(default=True)
* <b>cv</b>: 평가지표인 cross validation을 의미하며, 정수값을 넘겨줄수도 있고 사용자가 정의한 Fold를 넘겨줄 수도 있다.
* <b>verbose</b>: log 출력의 level을 조정한다. 숫자가 클수록 많은 log를 출력한다.

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

# load data
dataset = datasets.load_diabetes()
df=pd.DataFrame(data=dataset.data,columns=dataset.feature_names)
df['target'] = dataset.target
X=df.drop('target',axis=1)
y=df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1234)

params = {
    'n_estimators': [100,200],
    'mean_child_weight':[50,100],
    'learning_rate':[0.005,0.01,0.05],
    'colsample_bytree':[0.7,0.9],
    'nthread':[-1],
    'lambda':[0.1,1],
    'subsample': [0.6,0.8],
    'random_state' : [42]
}


model = XGBRegressor()

# 총 256가지 조합에 대해 5번의 cross validation을 수행했으므로 1280번의 fit을 수행한다.
clf = GridSearchCV(estimator=model,param_grid=params,scoring='neg_mean_absolute_error',n_jobs=-1,refit=True, cv=5,verbose=2)

clf.fit(X_train, y_train)
#최적의 hyperparameter
print(f"best parameters:{clf.best_params_} \nbest score:{clf.best_score_}")

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[16:36:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best parameters:{'colsample_bytree': 0.7, 'lambda': 0.1, 'learning_rate': 0.05, 'mean_child_weight': 50, 'n_estimators': 100, 'nthread': -1, 'random_state': 42, 'subsample': 0.8} 
best score:-46.18804808127328


## Optuna
ref: https://optuna.readthedocs.io/en/stable/index.html</br>
ref: https://ssoonidev.tistory.com/107<br>
ref: https://hojjimin-statistic.tistory.com/27<br>
ref: https://neptune.ai/blog/best-tools-for-model-tuning-and-hyperparameter-optimization

* 머신러닝에서 자동으로 파라미터를 최적화하기 위한 프레임워크이다.
* 파라미터의 범위를 지정해주거나, 파라미터가 될 수 있는 목록을 objective 함수에 지정하면 매 Trial마다 파라미터를 임의로 변경하면서 최적의 파라미터를 찾는다.
* define-by-run 방식의 api로 유연함을 자랑한다. define-by-run이란 연산 정의와 값 초기화가 동시에 되는 것으로,
연산그래프가 정의되는 동시에 연산이 이루어지는 것이다.

<font size=5> 사용 방법</font>

1. 매 시도(trial)마다 사용할 objective 함수 선언
* 1-(1) 사용할 파라미터들의 값 범위 지정
* 1-(2) 학습 모델 지정
* 1-(3) score 설정(mae, mse, accuracy등): 이 값이 함수의 return값이 된다.

2. create_study 함수로 새로운 study객체 생성.
* 2-(1) study 객체의 학습 방향 지정
* 2-(2) study 객체의 trial(시도) 횟수 지정

3. optimize 함수로 최적 파라미터 탐색 실행

<font size=5 >예시 데이터</font><br>
scikit-learn 패키지의 california 주택가격 데이터를 사용하였다.

<font size=5 ><b>Import Packages</b></font>

In [ ]:
!pip install optuna
!pip install xgboost

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

<font size=5 ><b> API: optuna.trial.Trial</b></font><br>
* ref: https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html#optuna.trial.Trial
<img src="https://drive.google.com/uc?export=download&id=1a5Jjnh2VboDrDpeSZHkOWaaKNPVWXUpx" width=700>

<font size=5 >methods</font><br>

* <b>optuna.trial.Trial.suggest_categorical(name, choices)</b> : 리스트 범위 내에서 값을 선택한다.
* <b>optuna.trial.Trial.suggest_int(name, low, high[, step, log])</b> : 범위 내에서 정수형 값을 선택한다.
* <b>optuna.trial.Trial.suggest_float(name, low, high, *[, step, log])</b> : 범위 내에서 소수형 값을 선택한다.
* <b>optuna.trial.Trial.suggest_uniform(name, low, high)</b> : 범위 내에서 균일분포 값을 선택한다.
* <b>optuna.trial.Trial.suggest_discrete_uniform(name, low, high, q)</b> : 범위 내에서 이산 균일분포 값을 선택한다.
*<b>optuna.trial.Trial.suggest_loguniform(name, low, high)</b> : 범위 내에서 로그 함수 값을 선택한다.<br>

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings(action='ignore')

# Data Load 
dataset = datasets.fetch_california_housing()
df=pd.DataFrame(data=dataset.data,columns=dataset.feature_names)
df['target'] = dataset.target

#학습 데이터 셋과 훈련 데이터 셋 생성
X=df.drop('target',axis=1)
y=df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1234)

# 각 trial마다 사용할 objective 함수 설정
def objective(trial: Trial, X, y, test):
    param = {
        'n_estimators' : trial.suggest_int('n_estimators', 100, 2000),
        'max_depth' : trial.suggest_int('max_depth', 8, 16),
        'min_child_weight' : trial.suggest_int('min_child_weight', 1, 300),
        'gamma' : trial.suggest_int('gamma', 1, 3),
        'learning_rate' :trial.suggest_float('learning_rate', 1e-3,5e-2),
        'colsample_bytree' : trial.suggest_discrete_uniform('colsample_bytree', 0.5, 1, 0.1),
        'nthread' : -1,
        'lambda' : trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha' : trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample' : trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0]),
        'random_state' : [1127]
    }
    
    # 학습 모델 생성
    model = XGBRegressor(**param)
    xgb_model = model.fit(X, y, verbose=True) # 학습 진행
    
    # 모델 성능 확인
    score = mean_squared_error(xgb_model.predict(X), y)
    
    return score



<font size=5 > <b>API: optuna.create_study </b></font><br>
* ref: https://optuna.readthedocs.io/en/stable/reference/generated/optuna.study.create_study.html#optuna.study.create_study
<img src="https://drive.google.com/uc?export=download&id=1I1K0_30zmFapexR-HYTm9AAIW_xtK0vz" height=60>

<font size=5 >parameter</font>
* <b>directions</b>: score값을 최대(maximize) 또는 최소(minimize)로 하는 방향 지정
* <b>sampler</b>: 샘플링 방식 지정. (default=TPESampler) 
* <b>pruner</b>: 성능이 좋지 않은 trial들의 조기 종료를 결정하는 pruner 객체이다. (default=Medianpruner)

In [ ]:
# objective함수에서 평가 기준을 mse로 설정했으므로, mse를 최소화시키는 방향으로 학습을 진행한다.
study = optuna.create_study(direction='minimize', sampler=TPESampler())

# n_trials 지정해주지 않으면 무한 반복
# optuna 안에 내장된 optimize 메서드를 이용해 최적 파라미터 탐색을 실행한다.
study.optimize(lambda trial : objective(trial, X, y, X_test), n_trials = 50)

print('Best trial : score {}, \nparams {} \nbesttrial {}'.format(study.best_trial.value, study.best_trial.params,study.best_trial))

[I 2022-03-19 10:17:00,082] A new study created in memory with name: no-name-3dff695a-8172-4b30-ad2e-50b74427152c


[10:17:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-03-19 10:17:08,063] Trial 0 finished with value: 0.12019461981113214 and parameters: {'n_estimators': 340, 'max_depth': 9, 'min_child_weight': 9, 'gamma': 2, 'learning_rate': 0.04519678131771279, 'colsample_bytree': 0.5, 'lambda': 0.015344978829737158, 'alpha': 0.0011847483058759843, 'subsample': 0.8}. Best is trial 0 with value: 0.12019461981113214.


[10:17:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-03-19 10:17:54,017] Trial 1 finished with value: 0.15727143991999712 and parameters: {'n_estimators': 1070, 'max_depth': 11, 'min_child_weight': 75, 'gamma': 2, 'learning_rate': 0.0066934611131868275, 'colsample_bytree': 1.0, 'lambda': 0.0011315192768346115, 'alpha': 1.7414614254977205, 'subsample': 1.0}. Best is trial 0 with value: 0.12019461981113214.


...

[10:51:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-03-19 10:53:00,895] Trial 48 finished with value: 0.07949326156037696 and parameters: {'n_estimators': 1852, 'max_depth': 13, 'min_child_weight': 7, 'gamma': 1, 'learning_rate': 0.003985773039666195, 'colsample_bytree': 0.5, 'lambda': 0.2557078974805324, 'alpha': 0.0022089459669698045, 'subsample': 0.8}. Best is trial 43 with value: 0.05180336445930823.


[10:53:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-03-19 10:53:43,897] Trial 49 finished with value: 0.16977369480176216 and parameters: {'n_estimators': 1679, 'max_depth': 14, 'min_child_weight': 282, 'gamma': 1, 'learning_rate': 0.013119257404382419, 'colsample_bytree': 0.5, 'lambda': 1.721612573331085, 'alpha': 0.005227660852236055, 'subsample': 0.8}. Best is trial 43 with value: 0.05180336445930823.


Best trial : score 0.05180336445930823, 
params {'n_estimators': 1808, 'max_depth': 14, 'min_child_weight': 1, 'gamma': 1, 'learning_rate': 0.018135469230736986, 'colsample_bytree': 1.0, 'lambda': 0.3993999415576441, 'alpha': 0.1206300703706598, 'subsample': 0.8} 
besttrial FrozenTrial(number=43, values=[0.05180336445930823], datetime_start=datetime.datetime(2022, 3, 19, 10, 45, 31, 105126), datetime_complete=datetime.datetime(2022, 3, 19, 10, 47, 19, 204945), params={'n_estimators': 1808, 'max_depth': 14, 'min_child_weight': 1, 'gamma': 1, 'learning_rate': 0.018135469230736986, 'colsample_bytree': 1.0, 'lambda': 0.3993999415576441, 'alpha': 0.1206300703706598, 'subsample': 0.8}, distributions={'n_estimators': IntUniformDistribution(high=2000, low=100, step=1), 'max_depth': IntUniformDistribution(high=16, low=8, step=1), 'min_child_weight': IntUniformDistribution(high=300, low=1, step=1), 'gamma': IntUniformDistribution(high=3, low=1, step=1), 'learning_rate': UniformDistribution(high=

<font size=5 > <b>API: optuna.visualization.plot_param_importances </b></font><br>
* ref: https://optuna.readthedocs.io/en/stable/reference/visualization/generated/optuna.visualization.plot_param_importances.html
<img src="https://drive.google.com/uc?export=download&id=1yXXXbtfVsFZ5UyvkGWyrPH7qp3XII-JX" height=60>

<font size=5 >parameter</font>
* <b>study</b>: 최적화가 이루어진 study객체
* <b>params</b>: 차트에 나타낼 parameter list

In [ ]:
optuna.visualization.plot_param_importances(study,params=['min_child_weight','max_depth','n_estimators','learning_rate','subsample','colsample_bytree']) # 파라미터 중요도 확인 그래프

<font size=5 > <b>API: optuna.visualization.plot_optimization_history </b></font><br>
* ref:https://optuna.readthedocs.io/en/stable/reference/visualization/generated/optuna.visualization.plot_optimization_history.html#optuna.visualization.plot_optimization_history
<img src="https://drive.google.com/uc?export=download&id=1dxvFoI2qTg65MXe1BMssS8tLwgLWWcQW" height=60>

<font size=5 >parameter</font>
* <b>study</b>: study 객체(최적화된 sutdy 객체를 입력하면 된다.)
* <b>params</b>: 차트에 나타낼 parameter list

In [ ]:
optuna.visualization.plot_optimization_history(study) # 최적화 과정 시각화